## Agents on Kubeflow 🤓

In this tutorial we will be training a reinforcement learning agent from the [tensorflow/agents](https://github.com/tensorflow/agents) project on Kubernetes using [Kubeflow](https://github.com/google/kubeflow).

The task the agent will be learning to perform is to operate a Kuka Robotics arm simulated in the OpenAI Gym Bullet Physics 'KukaBulletEnv-v0' environment. Feel free to [skip to the end](http://localhost:8888/notebooks/kubeflow-rl/apps/agents_ppo/demo.ipynb#Rendering-the-model) to see what this will look like!

### Setup

This demo assumes you've followed the setup steps provided in the readme. Beyond that we still have a few in-notebook steps to perform.

The first are to specify the bucket to which to write logs, the address of the image encapsulating our training job, and the path on the filesystem running this notebook where the root of kubeflow/examples can be found.

For the latter by default the container is built with these at /home/jovyan/kubeflow-examples.

In [198]:
TRAINING_IMAGE="gcr.io/kubeflow-rl/agents:tf1.4.1-0214-0810-acd8"
BUCKET="kubeflow-rl"
#EXAMPLES_ROOT="/home/jovyan/kubeflow-examples"
EXAMPLES_ROOT="/Users/cb/code/current/kubeflow/examples-agents0"
NAMESPACE="rl"
PROJECT="kubeflow-rl"
ZONE="us-east1-d"
CLUSTER="kubeflow-dev" # The name of your cluster here!
APP_NAME="agents"
APP_ROOT=os.path.join(EXAMPLES_ROOT, "agents")

#### Configuring GCP credentials

In order to run the commands in this notebook you'll have to have activated the right Google Cloud Platform credentials. This can be done in one of two ways. The first is to obtain a service account key file from the GCP console, upload it to your JupyterHub instance via the Jupyter upload dialog as described in the readme, and activate it using the following:

In [ ]:
!gcloud auth activate-service-account --key-file=[path to your service account key]

#### Verifying bucket

It may be confusing to debug error that result from running a training job attempting to log from a GCS bucket that doesn't exist or to which you don't have access. So let's verify that before going any further. The following will create the bucket if it doesn't already exist otherwise will fail indicating the bucket already exists. Importantly if the bucket already exists but we don't have access to it this will be stated as well.

In [167]:
!gsutil mb gs://{BUCKET}

Creating gs://kubeflow-rl/...
ServiceException: 409 Bucket kubeflow-rl already exists.


#### Configuring Kubernetes credentials

Having done so we are ready to obtain the credentials for the Kubernetes cluster on which we will be running the training tasks.

In [176]:
!gcloud container clusters --project={PROJECT} --zone={ZONE} get-credentials {CLUSTER}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for kubeflow-dev.


#### Configure ksonnet

Ksonnet is a tool to simplify configuration management for Kubernetes deployments which extends to making it easier to specify and re-configure distributed TensorFlow training jobs on Kubeflow.

We can initialize a new Ksonnet workspace and populate it with the necessary dependencies by running the following:

In [147]:
os.chdir(APP_ROOT)
!ks init app
os.chdir(os.path.join(APP_ROOT, "app"))
!ks registry add kubeflow github.com/kubeflow/kubeflow/tree/master/kubeflow
!ks pkg install kubeflow/tf-job

INFO  Using context 'gke_kubeflow-rl_us-east1-d_kubeflow-demo-1' from the kubeconfig file specified at the environment variable $KUBECONFIG
INFO  Creating a new app 'app' at path '/Users/cb/code/current/kubeflow/examples-agents0/agents/app'
INFO  Creating environment 'default' with namespace '', pointing at server at address 'https://35.196.93.70'
INFO  Generating environment metadata at path '/Users/cb/code/current/kubeflow/examples-agents0/agents/app/environments/default'
INFO  ksonnet app successfully created! Next, try creating a component with `ks generate`.
INFO  Retrieved 5 files


We can then stage relevant components from the config folder into place in this Ksonnet workspace as follows:

In [148]:
!cp {APP_ROOT}/config/*sonnet {APP_ROOT}/app/components/

If you're interesed you can see there is now a populated Ksonnet workspace directory which includes the components from our config directory:

In [149]:
!ls {APP_ROOT}/app

app.yaml     components   environments lib          vendor


In [150]:
!ls {APP_ROOT}/app/components

agents.jsonnet        params.libsonnet      tensorboard.libsonnet
agents_render.jsonnet tensorboard.jsonnet


Lastly we'll configure ksonnet to use a particular (the current) set of Kubernetes credentials by default:

In [ ]:
!ks env set default

For more information on Ksonnet check out their documentation [here](ksonnet.io).

### Building training image

##### Stage training code to GCS

Create a GCS bucket through which to stage code

In [199]:
!gsutil mb gs://{PROJECT}-builder

Creating gs://kubeflow-rl-builder/...


Package the example code

In [213]:
!mkdir -p {APP_ROOT}/../build/ && cd {APP_ROOT}/../ && tar czf build/{APP_NAME}.tgz {APP_NAME}

Stage to google cloud storage

In [225]:
BUILD_ID=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
TRAINER_TAG="gcr.io/%s/%s:%s" % (PROJECT, APP_NAME, BUILD_ID)

In [226]:
!gsutil cp {APP_ROOT}/../build/{APP_NAME}.tgz gs://{PROJECT}-builder/{APP_NAME}/{BUILD_ID}/
!gsutil ls gs://{PROJECT}-builder/{APP_NAME}/{BUILD_ID}/

Copying file:///Users/cb/code/current/kubeflow/examples-agents0/agents/../build/agents.tgz [Content-Type=application/x-tar]...
\ [1 files][  2.4 MiB/  2.4 MiB]                                                
Operation completed over 1 objects/2.4 MiB.                                      
gs://kubeflow-rl-builder/agents/0214-0810-2cc8/agents.tgz


##### Run the Argo container builder

In [ ]:
os.chdir(APP_ROOT)
!argo submit config/builder.yaml --namespace kubeflow \
    --parameter bundle=gs://{PROJECT}-builder/{APP_NAME}/{BUILD_ID}/{APP_NAME}.tgz \
    --parameter trainer-tag={TRAINER_TAG} \
    --parameter app-name=agents \
    --parameter project={PROJECT}

You can check on the progress of your build in the Argo UI via the following URL (provided you have `kubectl proxy` running on your localhost):

In [244]:
PROXY_PORT=8001
ARGO_NAMESPACE="kubeflow"
url=("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/argo-ui:80/".format(
    proxy_port=PROXY_PORT, namespace=ARGO_NAMESPACE))
print(url)

http://127.0.0.1:8001/api/v1/proxy/namespaces/kubeflow/services/argo-ui:80/


Which will look something like this as the build progresses:

![](builder.png)

### Training

The objective of the training phase is to learn the parameterization of our model that confers a high level of performance on the provided task. Here we'll launch and monitor a job.

#### Launching the TFJob

We'll use [ksonnet](https://ksonnet.io/) to parameterize and apply a TFJob configuration (i.e. run a job). Here you can change the image to be a custom job image, such as one built and deployed with build.sh, or use the one provided here if you only want to change parameters. Below we'll display the templated job YAML for reference.

In [ ]:
# Check your cluster and see if that matches one of the existing ksonnet environments
# You want the kubernetes master server to be the same as the server listed for the ks environment
!kubectl cluster-info
!ks env list

In [182]:
study = {"name": "replicated-env-comparison-demo",
         "experiments": [{"name": "kuka",
                          "image": TRAINING_IMAGE,
                          "env": "KukaBulletEnv-v0",
                          "num_replicas": 1}]
        }

STUDY_LOGS_ROOT = "gs://{0}/studies/{1}".format(BUCKET, study["name"])

print(STUDY_LOGS_ROOT)

gs://kubeflow-rl/studies/replicated-env-comparison-demo


In [183]:
import datetime
import uuid
import pprint

print("Preparing study: %s..." % study["name"])

for experiment in study["experiments"]:
    
    print("Preparing experiment: %s" % experiment["name"])
    
    # Get and set the job container image for this experiment
    IMAGE = experiment["image"]
    !ks param set agents image {IMAGE}

    # Set the gym learning environment on which to train
    ENVIRONMENT = experiment["env"]
    !ks param set agents env {ENVIRONMENT}

    # Set the algorithm and network part to use for policy and value networks
    !ks param set agents algorithm "agents.ppo.PPOAlgorithm"
    !ks param set agents network "agents.scripts.networks.feed_forward_gaussian"

    # Run in training mode with 30 CPU and 30 agents for 20M steps
    !ks param set agents run_mode train
    !ks param set agents num_cpu 30
    !ks param set agents num_agents 30
    !ks param set agents steps 15e6

    !ks param set agents update_every 60
    !ks param set agents eval_episodes 25

    for replica_id in range(experiment["num_replicas"]):

        # Construct a unique name for the training job based on experiment["name"]
        now=datetime.datetime.now()
        JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
        BASE_NAME = experiment["name"]
        TRAIN_JOB_NAME=BASE_NAME + "-" + JOB_SALT
        !ks param set agents name {TRAIN_JOB_NAME}

        # Construct a log dir path for this experiment
        LOG_DIR="{0}/{1}".format(STUDY_LOGS_ROOT, TRAIN_JOB_NAME)
        !ks param set agents log_dir {LOG_DIR}

        if "replicas" not in experiment:
            experiment["replicas"] = []
        experiment["replicas"].append({"log_dir": LOG_DIR})
        
        print("Preparing replica %s of %s for experiment %s" % (replica_id + 1, experiment["replicas"], experiment["name"]))
        !ks apply default -c agents

Preparing study: replicated-env-comparison-demo...
Preparing experiment: kuka
INFO  Parameter 'image' successfully set to '"gcr.io/kubeflow-rl/agents:tf1.4.1-0214-0810-acd8"' for component 'agents'
INFO  Parameter 'env' successfully set to '"KukaBulletEnv-v0"' for component 'agents'
INFO  Parameter 'algorithm' successfully set to '"agents.ppo.PPOAlgorithm"' for component 'agents'
INFO  Parameter 'network' successfully set to '"agents.scripts.networks.feed_forward_gaussian"' for component 'agents'
INFO  Parameter 'run_mode' successfully set to '"train"' for component 'agents'
INFO  Parameter 'num_cpu' successfully set to '30' for component 'agents'
INFO  Parameter 'num_agents' successfully set to '30' for component 'agents'
INFO  Parameter 'steps' successfully set to '15e6' for component 'agents'
INFO  Parameter 'update_every' successfully set to '60' for component 'agents'
INFO  Parameter 'eval_episodes' successfully set to '25' for component 'agents'
INFO  Parameter 'name' successfull

In [180]:
study

{'experiments': [{'env': 'KukaBulletEnv-v0',
   'image': 'gcr.io/kubeflow-rl/agents:tf1.4.1-0213-1451-e98b',
   'name': 'kuka',
   'num_replicas': 4,
   'replicas': [{'log_dir': 'gs://kubeflow-rl/studies/replicated-env-comparison-demo/kuka-0214-0808-cd2e'},
    {'log_dir': 'gs://kubeflow-rl/studies/replicated-env-comparison-demo/kuka-0214-0808-b896'},
    {'log_dir': 'gs://kubeflow-rl/studies/replicated-env-comparison-demo/kuka-0214-0808-fd9e'},
    {'log_dir': 'gs://kubeflow-rl/studies/replicated-env-comparison-demo/kuka-0214-0808-e56d'}]}],
 'name': 'replicated-env-comparison-demo'}

Now we can list tfjobs and see that a job has been created.

In [159]:
!kubectl get tfjobs -n rl

NAME                  AGE
kuka-0213-1552-5db9   9s
kuka-0213-1552-e612   12s
kuka-0213-1553-1234   6s
kuka-0213-1553-14b6   4s


#### Monitoring training

The IDs, status, and other metadata of pods involved in the training job can be displayed using the following:

In [161]:
!kubectl get pods -n rl --show-all

NAME                                      READY     STATUS              RESTARTS   AGE
ambassador-d96567548-6gvv5                0/2       ContainerCreating   0          48s
ambassador-d96567548-hg6zj                0/2       ContainerCreating   0          48s
ambassador-d96567548-nhkl2                0/2       ContainerCreating   0          48s
kuka-0213-1552-5db9-master-npy7-0-cqq97   0/1       ContainerCreating   0          14s
kuka-0213-1552-e612-master-kude-0-nssc8   0/1       ContainerCreating   0          17s
kuka-0213-1553-1234-master-qqnr-0-grmwx   0/1       Pending             0          11s
kuka-0213-1553-14b6-master-o2jk-0-mj9vj   0/1       Pending             0          9s
tf-hub-0                                  0/1       ContainerCreating   0          49s
tf-job-dashboard-97b55fb8c-4jvk4          1/1       Running             0          49s
tf-job-operator-8679dd4fb5-cnbv6          1/1       Running             0          49s


In [162]:
TRAIN_JOB_NAME

'kuka-0213-1553-14b6'

Obtain the ID of the master pod and print logs

In [ ]:
import subprocess
master_pod = subprocess.check_output(["kubectl", "-n", NAMESPACE, "get", "pods", "--selector=tf_job_name=" + TRAIN_JOB_NAME,
                                      "-o", "jsonpath='{.items[*].metadata.name}'"]).decode("utf-8")
print(master_pod)
!kubectl logs -n {NAMESPACE} {master_pod}

##### Launching tensorboard

The following will create a tensorboard deployment using the STUDY_LOGS_ROOT as the source of tensorboard logs.

In [164]:
HPARAM_SET="tboard"
now=datetime.datetime.now()
JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
TBOARD_JOB_NAME=HPARAM_SET + "-" + JOB_SALT
SECRET_NAME="gcp-credentials"
SECRET_FILE_NAME="secret.json"
NAMESPACE="rl"

!ks param set tensorboard name {TBOARD_JOB_NAME}
!ks param set tensorboard namespace {NAMESPACE}
!ks param set tensorboard log_dir {STUDY_LOGS_ROOT}
!ks param set tensorboard secret {SECRET_NAME}
!ks param set tensorboard secret_file_name {SECRET_FILE_NAME}
!ks show default -c tensorboard

!ks apply default -c tensorboard

INFO  Parameter 'name' successfully set to '"tboard-0213-1556-a4c3"' for component 'tensorboard'
INFO  Parameter 'namespace' successfully set to '"rl"' for component 'tensorboard'
INFO  Parameter 'log_dir' successfully set to '"gs://kubeflow-rl/studies/replicated-env-comparison-demo"' for component 'tensorboard'
INFO  Parameter 'secret' successfully set to '"gcp-credentials"' for component 'tensorboard'
INFO  Parameter 'secret_file_name' successfully set to '"secret.json"' for component 'tensorboard'
---
apiVersion: apps/v1beta1
kind: Deployment
metadata:
  name: tboard-0213-1556-a4c3-tb
  namespace: rl
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: tensorboard
        tb-job: tboard-0213-1556-a4c3
      name: tboard-0213-1556-a4c3
      namespace: rl
    spec:
      containers:
      - command:
        - /usr/local/bin/tensorboard
        - --logdir=gs://kubeflow-rl/studies/replicated-env-comparison-demo
        - --port=80
        env:
        - name: GOOGLE

##### Connecting to Tensorboard

To connect to TensorBoard from a browser running on your local machine start the kubernetes proxy with `kubectl proxy` then access TensorBoard via the url given by evaluating the next cell

In [165]:
PROXY_PORT=8001
# url=("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/{service_name}:80/".format(
#     proxy_port=PROXY_PORT, namespace=NAMESPACE, service_name=TRAIN_JOB_NAME + "-tb"))
url=("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/{service_name}:80/".format(
    proxy_port=PROXY_PORT, namespace=NAMESPACE, service_name=TBOARD_JOB_NAME + "-tb"))
print(url)

http://127.0.0.1:8001/api/v1/proxy/namespaces/rl/services/tboard-0213-1556-a4c3-tb:80/


Below is a screen grab from Tensorboard of the mean_score for four identically parameterized workers training on the Kuka environment:

![](tboard_mean_score.png)

As you can see there is variability both between and within workers but some still seem to reach stably high performance.

## Rendering!

Let's take a look at the trained model performing the task!!

#### Initiating render jobs

Here we'll create a render job for each of the experiments in our study. This will generate multiple MP4 videos of the agent performing the task for each replica run with renders landing in ${LOG_DIR}/render/[some_unique_subdir_name].

In [ ]:
import datetime
import uuid
import os

os.chdir(os.path.join(APP_ROOT, "app"))

for experiment in study["experiments"]:
    for replica in experiment["replicas"]:
        
        LOG_DIR = replica["log_dir"]
        IMAGE = experiment["image"]
        
        now=datetime.datetime.now()

        JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
        RENDER_JOB_NAME="render-" + JOB_SALT
        !ks param set agents_render name {RENDER_JOB_NAME}

        !ks param set agents_render log_dir {LOG_DIR}
        !ks param set agents_render image {IMAGE}
        
        !ks apply default -c agents_render

It will take about 15min for each render to run. We can see whether the renders are complete by checking the third column in the following:

In [ ]:
!kubectl get jobs -n rl --show-all

First let's get the GCS path of the directory containing renders for an experiment of interest. For a reminder, here's the structure of our study:

In [ ]:
study

Let's take a look at the renders for the first replica of the first experiment. First let's take a look at the log dir and render/ subdir of that experiment:

In [ ]:
kuka_experiment_logdir = study["experiments"][0]["replicas"][0]["log_dir"]
!gsutil ls {kuka_experiment_logdir}
!gsutil ls {kuka_experiment_logdir}/render

Now let's specify exactly which render directory from which to pull a render for inspection. In my case above the render dir would be "gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/render/0208-2228-7d1d/":

In [170]:
RENDER_DIR="gs://[your bucket]/studies/[study name]/[job name]/render/[render id]"

Now let's take a look at one of those renders:

In [ ]:
!mkdir -p /tmp/agents-render
!gsutil cp `gsutil ls {RENDER_DIR} | grep mp4 | head -n1` /tmp/agents-render/render.mp4

#### Inspecting the result

When the job is complete there will be a subdirectory of the log dir named "render" with a number of short videos of episodes of the agent performing the grasping task. Here's an example of what one of those looks like in a well-trained model.

In [ ]:
import io
import base64
from IPython.display import HTML

mp4_path = '/tmp/agents-render/render.mp4'

video = io.open(mp4_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

### Great job! 🎉🎉🎉

If this is your first time working with these technologies you might be interested in some suggestions of good next steps. Here are some ideas:
- Try training with some other learning environments (from the ID fields [here](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/__init__.py)) and tweet your results! E.g.
    - RacecarBulletEnv-v0
    - MinitaurBulletDuckEnv-v0
    - HalfCheetahBulletEnv-v0
- Take a shot at implementing your own gym learning environment and repeat the above.